In [14]:
import torch
import tqdm
from typing import Optional
from sklearn.metrics import log_loss, ndcg_score, roc_curve, auc, recall_score, precision_score, f1_score
from rs.models.xdfm import ExtremeDeepFactorizationMachineModel
from rs.datasets.avanzu import AvanzuDatasetBuilder
import logging
import torch.nn as nn
from rs.train_avanzu import get_model, get_baseline_model
import seaborn as sns
import pandas as pd
import numpy as np
from rs.models.modelwarapper import PredictionsUtils

In [2]:
CHECKPINT_PATH = '/rs/RSproject/rs/chkpt/avanzu_model_checkpoint_v1.pt'
DATASET_PATH = '/rs/datasets/avazu-ctr-prediction'

In [3]:
batch_size = 512
device = 'cuda:0' if torch.cuda.is_available()  else 'cpu'
device, batch_size

dataset = AvanzuDatasetBuilder.create_dataset(DATASET_PATH,
                                                 batch_size= batch_size)

model_state_dict = torch.load(CHECKPINT_PATH)['model_state_dict']
xdf_model = get_model(dataset)
xdf_model.load_state_dict(model_state_dict)
xdf_model.eval()
xdf_model.to(device)

read
loaded


ExtremeDeepFactorizationMachineModel(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(23681188, 8)
  )
  (cin): CompressedInteractionNetwork(
    (conv_layers): ModuleList(
      (0): Conv1d(484, 8, kernel_size=(1,), stride=(1,))
      (1): Conv1d(176, 8, kernel_size=(1,), stride=(1,))
    )
    (fc): Linear(in_features=16, out_features=1, bias=True)
  )
  (mlp): MultiLayerPerceptron(
    (mlp): Sequential(
      (0): Linear(in_features=176, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=8, out_features=8, bias=True)
      (5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=8, out_features=1, bias=True)
    )
  )
  (linear): FeaturesLinear(
    (fc): Embedding(23681188, 1)
  )
)

In [4]:
baseline_xgb_model = get_baseline_model()

In [5]:
xdfm_predictions_df = PredictionsUtils.get_xdfm_avanzu_predictions(xdf_model, DATASET_PATH, device)
xdfm_predictions_df.sample(1)

read
loaded


100%|██████████| 63171/63171 [00:41<00:00, 1536.51it/s]


,target,prediction,site_id
46457,1.0,0.274234,4


In [6]:
xgb_predictions_df = PredictionsUtils.get_xgboost_avanzu_predictions(baseline_xgb_model, DATASET_PATH)
xgb_predictions_df.sample(2)

read
loaded


,target,prediction,site_id
1741406,0.0,0.0,7
781421,1.0,0.0,4


In [17]:
def calc_metric(predictions_df, metric_func):
    return metric_func(predictions_df['target'], predictions_df['prediction'])

def calc_auc(predictions_df):
    
    fpr, tpr, _ = roc_curve(predictions_df['target'], predictions_df['prediction'])
    
    return auc(fpr, tpr)

#, , 

def calc_metrics(predictions_df, model_name):
    
    model_log_loss_score = calc_metric(predictions_df, log_loss)
    auc_score = calc_auc(predictions_df)
    
    return pd.DataFrame([(model_name, model_log_loss_score, auc_score, )],
                       columns=['model_name','log_loss', 'auc_score'] )



In [ ]:
calc_metrics(xdfm_predictions_df, 'xdfm')

In [ ]:
calc_metrics(xgb_predictions_df, 'xgb')